In [1]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import spacy
from optimized_anchor import anchor_text, anchor_base
import pickle
import myUtils
from myUtils import *
from dataset.dataset_loader import *
import datetime
%load_ext line_profiler

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# can be sentiment/spam/offensive
dataset_name = 'sentiment'
text_parser, label_parser, ds_train, ds_val = get_dataset(dataset_name)

Number of tokens in training samples: 6407
Number of tokens in training labels: 2


In [3]:
train, train_labels, test, test_labels, anchor_examples = preprocess_examples(ds_train)

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
ignored = get_ignored(anchor_examples)
normal_occurences = get_occurences(anchor_examples)
bg = BestGroup('check', normal_occurences, filter_anchors = False, desired_optimize = False)

## notice!

In [6]:
ignored = []

In [7]:
examples = anchor_examples[1:3]

In [8]:
!nvidia-smi

Sun Aug 14 18:38:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:AF:00.0 Off |                  Off |
| 33%   32C    P8    17W / 260W |   9527MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
optimize = True
anchor_text.AnchorText.set_optimize(optimize)
explainer = anchor_text.AnchorText(nlp, ['positive', 'negative'], use_unk_distribution=False)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [11]:
from transformers import AutoTokenizer
model = torch.jit.load(f'models/tinybert/{dataset_name}/traced.pt').to(device)
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', use_fast = False)
myUtils.model = model
myUtils.tokenizer = tokenizer

In [ ]:
tokenized = explainer.tg.bert_tokenizer(["i [MASK] to [MASK] party yesterday"], 
          add_special_tokens=True, return_tensors="pt", padding=True)["input_ids"]

In [ ]:
tokenized_seperate = [explainer.tg.bert_tokenizer.encode("i went to a party yesterday", 
          add_special_tokens=True) for _ in range(100)]
tokenized_seperate = [torch.tensor([t], device=device) for t in tokenized_seperate]

In [ ]:
x = np.array([1,2,3,4,5,6])

In [ ]:
tokenizer.decode([  101,   103,  2049,  2219, 13109, 28819,  7999,  2126,  1010,
        2009,  4152,  2000,  2017,  1012,  2074,  2066,  1045, 18259,
        2100,   102])

In [12]:
#torch._C._jit_set_texpr_fuser_enabled(False)
my_utils = TextUtils(examples, test, explainer, predict_sentences, ignored, f"profile.pickle", optimize=optimize)
myUtils.model = model
myUtils.tokenizer = tokenizer
anchor_base.AnchorBaseBeam.best_group = bg
set_seed()
%prun -s cumtime -T check/profile.txt my_utils.compute_explanations(list(range(len(examples))))

number 0
[['in', 'its', 'own', 'fl', '##ound', '##ering', 'way', ',', 'it', 'gets', 'to', 'you', '.', 'just', 'like', 'i', '##gb', '##y']]
tensor([[0.0318, 0.9682]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
[array(['ब', ',', '##ा', '##ா', 'க', '##テ', ',', ',', 'த', '##ت', '##ᅦ',
       '##ת', '##ன', '[', '##ு', '##ர', '##ד', '##ト'], dtype='<U80')]
tensor([[0.2758, 0.7242]], device='cuda:0')
[array(['earth', '##enes', 'more', 'fl', '##ound', '##ering', ':', 'into',
       '##リ', '##ᆯ', 'little', 'honey', '##ի', 'parties', 'like',
       'diamond', 'music', '##tery'], dtype='<U80'), array(['<', '##ᅢ', 'slightly', 'fl', '##ound', '##ering', '##ᅢ', ',',
       'bwv', '##ই', '##コ', '##া', 'on', '##র', '##ম', '##ன', '##ク',
       '##த'], dtype='<U80'), array(['in', 'ordinary', '##ese', '##ent', '##ath', '##ering', 'pig',
       '##ization', 'it', 'makes', 'to', ':', 'like', '##hedron', 'like',
       'ge', 'and', '.'], dtype='<U80'), array(['pl', '##δ', '##cter', 'fl', '##inn', '##no', '

         1765114 function calls (1676831 primitive calls) in 3.282 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     83/1    0.001    0.000    3.284    3.284 {built-in method builtins.exec}
        1    0.000    0.000    3.284    3.284 <string>:1(<module>)
        1    0.000    0.000    3.283    3.283 myUtils.py:292(compute_explanations)
        2    0.012    0.006    3.282    1.641 myUtils.py:267(get_exp)
        2    0.000    0.000    3.270    1.635 anchor_text.py:265(explain_instance)
        2    0.001    0.000    1.986    0.993 anchor_base.py:255(anchor_beam)
       34    0.004    0.000    1.980    0.058 anchor_text.py:225(sample_fn)
      137    1.874    0.014    1.876    0.014 module.py:1096(_call_impl)
       34    0.001    0.000    1.738    0.051 myUtils.py:24(predict_sentences)
       69    0.001    0.000    1.391    0.020 anchor_text.py:153(probs)
        2    0.000    0.000    1.283    0.641 anchor_text.py:

In [ ]:
###### my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored,f"profile.pickle")
#%lprun -s -m modified_anchor.anchor_text -m modified_anchor.anchor_base -m myUtils -T profile.txt  my_utils.compute_explanations(list(range(len(anchor_examples))))

In [ ]:
print(datetime.datetime.now())